# Building Effective Agents (with Pydantic AI)

Examples for the agentic workflows discussed in
[Building Effective Agents](https://www.anthropic.com/research/building-effective-agents)
by [Erik Schluntz](https://github.com/eschluntz) and [Barry Zhang](https://github.com/ItsBarryZ)
of Anthropic, inspired, ported and adapted from the
[code samples](https://github.com/anthropics/anthropic-cookbook/tree/main/patterns/agents)
by the authors using [Pydantic AI](https://ai.pydantic.dev/).

## Orchestrator - Workers
Examples copied from [Intellectronica - Building Effective Agents with Pydantic AI](https://github.com/intellectronica/building-effective-agents-with-pydantic-ai)

In [5]:
%pip install -r requirements.txt
from IPython.display import clear_output ; clear_output()

In [6]:
from util import initialize, show
AI_MODEL = initialize()

import asyncio
import json
from typing import List, Dict

from pydantic import BaseModel, Field
from pydantic_ai import Agent

Available AI models:
['openai:gpt-4o', 'openai:gpt-4o-mini']

Using AI model: openai:gpt-4o


### Workflow: Orchestrator - Workers

An orchestrator is an agent that determines what work needs to be done
and divides it so that it can be delegated to multiple workers. Once
the work plan is ready, we run each task by a separate worker in parallel
and eventually collect the results from all workers.

> <img src="https://ai.pydantic.dev/img/pydantic-ai-dark.svg" style="height: 1em;" />
> The schemas defined by Pydantic are translated into JSON schema, which is passed on
> to the model API for defining how the result should be structured. Each API handles
> this a bit differentls, but the details are abstracted by the library. In general,
> some complexity and nesting of types is usually OK. In this case, we define a `Task`
> type, and return a list of tasks as part of `OrchestratorResponse`.
>
> Agents in Pydantic AI have some fixed definitions, like their `system_prompt`, and
> can be used to execute multiple runs, each with its own prompt. Here we define a
> worker agent instance, which can execute each task definition.

In [7]:
class Task(BaseModel):
    type: str = Field(..., description=(
        'The type of minion task approach. '
        'For example: "enthusiastic", "technical", "banana-focused", "gadget-oriented", ...'))
    description: str = Field(
        ...,
        description='Clear description for executing this minion task.'
    )


class OrchestratorResponse(BaseModel):
    analysis: str = Field(..., description=(
        'Explain your understanding of the minion task and which variations '
        'would be valuable for Gru\'s operations. Focus on how each approach serves '
        'different aspects of minion management or evil schemes.'
    ))
    tasks: List[Task] = Field(..., description="List of minion tasks")


async def orchestrate(task: str) -> Dict:
    """Process minion task by breaking it down and running subtasks in parallel."""

    orchestrator_agent = Agent(
        AI_MODEL,
        system_prompt=(
            'You are Gru\'s master orchestrator for minion operations. ',
            'Analyze this minion-related task and break it down into ',
            '2-3 distinct approaches that different types of minions could handle.',
        ),
        output_type=OrchestratorResponse,
    )
    orchestrator_response = await orchestrator_agent.run(task)
    
    analysis = orchestrator_response.output.analysis
    tasks = orchestrator_response.output.tasks
    
    show('', title='Minion Orchestrator Output')
    show(analysis, title='Analysis')
    show([task.model_dump() for task in tasks], title='Minion Tasks')
    
    # Process all the minion tasks in parallel and collect results
    worker_agent = Agent(
        AI_MODEL,
        system_prompt='You are a specialized minion worker. Generate content based on the minion task specification. Use minion language and enthusiasm where appropriate.',
    )
    worker_responses = await asyncio.gather(*[
        worker_agent.run(json.dumps(
            {'original_minion_task': task} | task_info.model_dump()
        ))
        for task_info in tasks
    ])

    for task, response in zip(tasks, worker_responses):
        show(response.output, title=f"Minion Worker Result ({task.type})")

In [8]:
await orchestrate(
    'Create a comprehensive training manual for new minions joining Gru\'s evil operations, covering banana breaks, gadget safety, and proper "BELLO!" etiquette.'
)


Minion Orchestrator Output
--------------------------


Analysis
--------

Creating a comprehensive training manual for new minions involves focusing on distinct areas vital for their integration and efficient functioning within Gru's operations. By tailoring the training approaches to cater to different minion skill sets and personalities, Gru can ensure a well-rounded workforce ready to tackle varied aspects of his schemes. Here are three different approaches: 'enthusiastic,' 'technical,' and 'etiquette-focused.' Each serves a unique area of minion management, allowing for specialization and strengthening overall capabilities.


Minion Tasks
------------

[{'description': 'Energetically introduce new minions to the joy of being part '
                 "of Gru's team. Cover basic operations with enthusiasm and "
                 'excitement, ensuring minions engage with content related to '
                 'banana breaks, making it entertaining and motivating. Use '
                